In [1]:
import transformers
from transformers import CLIPConfig, CLIPModel, CLIPProcessor, CLIPImageProcessor, CLIPTokenizerFast
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
import random
import math
import scipy.io as sio
import nibabel as nib
from pathlib import Path
from gensim.models import Word2Vec
import re

In [119]:
mat_contents = sio.loadmat('subject_1.mat')
words = mat_contents["words"][0]
list_words = []
run = []
length = 4

context = 3
scan = []

def get_next_four(start_index, word_list):
    next_four = []
    end_index = 0
    a = 0
    end_of_run = False
    for x in range(3,-1,-1):
        if start_index-x >= 0:
            if word_list[start_index-x][2][0][0] <= 0.5:
                word = word_list[start_index-x][0][0][0][0]
                while('"' in word):
                    word = word.replace('"','')
                next_four.append(word)
            else:
                a = x
                end_of_run = True
                break
    if a == 0:
        a = 3
    end_index = start_index + a +1
    return [" ".join(next_four),end_index,end_of_run]

def ramp_up(goal_context,index):
    scan = []
    b = index
    for a in range(1,goal_context):
        a,b,c = get_next_four(b,words)
        scan.append(a)
    to_return = []
    start = 0
    for a in range(1,goal_context):
        if a == 1:
            to_return.append(scan[start])
        else:
            to_return.append(" ".join(scan[start:start+a]))

    #print(to_return)    
    return to_return

index = 3 
end_of_run = True
while(index < len(words)):
    scan = []
    b = index
    if end_of_run:
        a = ramp_up(context,index)
        for x in a:
            list_words.append(x)
        end_of_run = False
    else:
        
        for y in range(context):
            if not end_of_run:
                a, b, end_of_run = get_next_four(b,words)
                scan.append(a)
        if not end_of_run:
            index += length
        else:
            index = b
        #print(scan)
        #print(scan)
        list_words.append(" ".join(scan))

        



['Harry had never believed', 'he would meet a', 'boy he hated more']
['he would meet a', 'boy he hated more', 'than Dudley, but that']
['boy he hated more', 'than Dudley, but that', 'was before he met']
['than Dudley, but that', 'was before he met', 'Draco Malfoy. Still, first-year']
['was before he met', 'Draco Malfoy. Still, first-year', 'Gryffindors only had Potions']
['Draco Malfoy. Still, first-year', 'Gryffindors only had Potions', 'with the Slytherins, so']
['Gryffindors only had Potions', 'with the Slytherins, so', "they didn't have to"]
['with the Slytherins, so', "they didn't have to", 'put up with Malfoy']
["they didn't have to", 'put up with Malfoy', 'much. Or at least,']
['put up with Malfoy', 'much. Or at least,', "they didn't until they"]
['much. Or at least,', "they didn't until they", 'spotted a notice pinned']
["they didn't until they", 'spotted a notice pinned', 'up in the Gryffindor']
['spotted a notice pinned', 'up in the Gryffindor', 'common room that made']
['up 

In [120]:
print(len(list_words))
print(list_words)

1299
['Harry had never believed', 'Harry had never believed he would meet a', 'Harry had never believed he would meet a boy he hated more', 'he would meet a boy he hated more than Dudley, but that', 'boy he hated more than Dudley, but that was before he met', 'than Dudley, but that was before he met Draco Malfoy. Still, first-year', 'was before he met Draco Malfoy. Still, first-year Gryffindors only had Potions', 'Draco Malfoy. Still, first-year Gryffindors only had Potions with the Slytherins, so', "Gryffindors only had Potions with the Slytherins, so they didn't have to", "with the Slytherins, so they didn't have to put up with Malfoy", "they didn't have to put up with Malfoy much. Or at least,", "put up with Malfoy much. Or at least, they didn't until they", "much. Or at least, they didn't until they spotted a notice pinned", "they didn't until they spotted a notice pinned up in the Gryffindor", 'spotted a notice pinned up in the Gryffindor common room that made', 'up in the Gryffin

In [68]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

inputs = processor(text=list_words, images=image, return_tensors="pt", padding=True)
outputs = model(**inputs)
# text_embeds is in the same space as the given image
text_embeds = outputs['text_embeds']

# pooler_output is just the text embedding 
# This is what we want to use I think 
pooler_output = outputs['text_model_output']['pooler_output']

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [69]:
print(pooler_output)
#print(text_embeds)

tensor([[-1.1388, -0.6321,  0.6394,  ...,  0.1586, -0.4268, -2.5951],
        [-0.5995, -0.7232,  2.0650,  ...,  0.1401, -0.8075, -1.7551],
        [ 0.5946, -0.6241,  1.0181,  ..., -0.2068, -0.2833, -0.7693],
        ...,
        [ 0.2055,  0.7853,  0.6895,  ..., -0.9441, -0.0887, -1.6115],
        [-0.4824,  0.3981, -0.9851,  ..., -1.8415,  0.1985, -0.1415],
        [-0.7414,  0.4709,  0.1839,  ..., -1.0032,  2.0105, -0.5915]],
       grad_fn=<IndexBackward0>)
